In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
input_data_folder = "../../../Master_Thesis_Project/prepared_data/"
output_data_folder = "../experiments/logdata/"
filename = "bpi20.csv"

In [3]:
case_id_col = "Case ID"
activity_col = "Activity"
resource_col = "Resource"
timestamp_col = "Complete Timestamp"

In [4]:
freq_threshold = 10

In [5]:
# features for classifier
static_cat_cols = ["BudgetNumber", "DeclarationNumber"]
static_num_cols = ["Amount"]
dynamic_cat_cols =[activity_col,resource_col,"id", "Role"]
dynamic_num_cols = []

In [6]:
static_cols = static_cat_cols + static_num_cols + [case_id_col]
dynamic_cols = dynamic_cat_cols + dynamic_num_cols + [timestamp_col]
cat_cols = dynamic_cat_cols + static_cat_cols

In [7]:
def extract_timestamp_features(group):
    
    group = group.sort_values(timestamp_col, ascending=False, kind='mergesort')
    
    start_date = group[timestamp_col].iloc[-1]
    end_date = group[timestamp_col].iloc[0]

    
    tmp = group[timestamp_col] - group[timestamp_col].shift(-1)
    tmp = tmp.fillna(pd.Timedelta(minutes=0))
    group["duration"] = tmp.apply(lambda x: float(x / pd.Timedelta(minutes=1))) # m is for minutes
    
    group["month"] = group[timestamp_col].dt.month
    group["weekday"] = group[timestamp_col].dt.weekday
    group["hour"] = group[timestamp_col].dt.hour
    
    elapsed = group[timestamp_col] - start_date
    elapsed = elapsed.fillna(pd.Timedelta(seconds=0))
    group["elapsed"] = elapsed.apply(lambda x: float(x / pd.Timedelta(seconds=1)))  # s is for seconds

    remtime = end_date - group[timestamp_col]
    remtime = remtime.fillna(pd.Timedelta(seconds=0))
    group["remtime"] = remtime.apply(lambda x: float(x /pd.Timedelta(seconds=1))) # s is for seconds
    
    return group

In [8]:
def get_open_cases(date):
    return sum((dt_first_last_timestamps["start_time"] <= date) & (dt_first_last_timestamps["end_time"] > date))


In [9]:
data = pd.read_csv(os.path.join(input_data_folder,filename), sep=",")

In [10]:
data = data[static_cols + dynamic_cols]

In [11]:
data.head()

,BudgetNumber,DeclarationNumber,Amount,Case ID,Activity,Resource,id,Role,Complete Timestamp
0,budget 86566,declaration number 86792,26.851205,declaration 86791,Declaration SUBMITTED by EMPLOYEE,STAFF MEMBER,st_step 86794_0,EMPLOYEE,2017-01-09 09:49:50+01:00
1,budget 86566,declaration number 86792,26.851205,declaration 86791,Declaration FINAL_APPROVED by SUPERVISOR,STAFF MEMBER,st_step 86793_0,SUPERVISOR,2017-01-09 11:27:48+01:00
2,budget 86566,declaration number 86792,26.851205,declaration 86791,Request Payment,SYSTEM,dd_declaration 86791_19,UNDEFINED,2017-01-10 09:34:44+01:00
3,budget 86566,declaration number 86792,26.851205,declaration 86791,Payment Handled,SYSTEM,dd_declaration 86791_20,UNDEFINED,2017-01-12 17:31:22+01:00
4,budget 86566,declaration number 86796,182.464172,declaration 86795,Declaration SUBMITTED by EMPLOYEE,STAFF MEMBER,st_step 86798_0,EMPLOYEE,2017-01-09 10:26:14+01:00


In [12]:
# add features extracted from timestamp
data[timestamp_col] = pd.to_datetime(data[timestamp_col],utc=True)
data = data.groupby(case_id_col,as_index=False).apply(extract_timestamp_features)

In [13]:
data.head()

BudgetNumber          DeclarationNumber      Amount  \
0 12792  budget 86566  declaration number 100001  600.844121   
  12791  budget 86566  declaration number 100001  600.844121   
  12790  budget 86566  declaration number 100001  600.844121   
  12789  budget 86566  declaration number 100001  600.844121   
  12788  budget 86566  declaration number 100001  600.844121   

                    Case ID                                  Activity  \
0 12792  declaration 100000                           Payment Handled   
  12791  declaration 100000                           Request Payment   
  12790  declaration 100000  Declaration FINAL_APPROVED by SUPERVISOR   
  12789  declaration 100000    Declaration APPROVED by ADMINISTRATION   
  12788  declaration 100000         Declaration SUBMITTED by EMPLOYEE   

             Resource                        id            Role  \
0 12792        SYSTEM  dd_declaration 100000_20       UNDEFINED   
  12791        SYSTEM  dd_declaration 100000_19       UNDEFINED   
  12790  STAFF MEMBER          st_step 100002_0      SUPERVISOR   
  12789  STAFF MEMBER          st_step 100003_0  ADMINISTRATION   
  12788  STAFF MEMBER          st_step 100004_0        EMPLOYEE   

               Complete Timestamp      duration  month  weekday  hour  \
0 12792 2018-02-12 16:31:20+00:00   4608.516667      2        0    16   
  12791 2018-02-09 11:42:49+00:00   1543.733333      2        4    11   
  12790 2018-02-08 09:59:05+00:00   1500.316667      2        3     9   
  12789 2018-02-07 08:58:46+00:00  11558.650000      2        2     8   
  12788 2018-01-30 08:20:07+00:00      0.000000      1        1     8   

           elapsed    remtime  
0 12792  1152673.0        0.0  
  12791   876162.0   276511.0  
  12790   783538.0   369135.0  
  12789   693519.0   459154.0  
  12788        0.0  1152673.0

In [14]:
data = data.droplevel(0)

In [15]:
def assign_label(group):
    relevant_activity_idxs = np.where(group[activity_col] == "Payment")[0]
    if len(relevant_activity_idxs) > 0:
        cut_idx = relevant_activity_idxs[0]
        group[label_col] = pos_label
        return group[:cut_idx]
    else:
        group[label_col] = neg_label
        return group

In [16]:
# assign labels
label_col = "label"
pos_label = "deviant"
neg_label = "regular"
#data = data.sort_values(timestamp_col, ascending=True, kind='mergesort').groupby(case_id_col).apply(assign_label)

In [17]:
# add features extracted from timestamp
# print("Assigning labels...")
# cut traces before relevant activity happens
# data = data.sort_values([case_id_col,timestamp_col], ascending=True).groupby(case_id_col).apply(assign_label)
data.sort_values([case_id_col,timestamp_col], axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')

In [18]:
data.head()

,BudgetNumber,DeclarationNumber,Amount,Case ID,Activity,Resource,id,Role,Complete Timestamp,duration,month,weekday,hour,elapsed,remtime
12788,budget 86566,declaration number 100001,600.844121,declaration 100000,Declaration SUBMITTED by EMPLOYEE,STAFF MEMBER,st_step 100004_0,EMPLOYEE,2018-01-30 08:20:07+00:00,0.000000,1,1,8,0.0,1152673.0
12789,budget 86566,declaration number 100001,600.844121,declaration 100000,Declaration APPROVED by ADMINISTRATION,STAFF MEMBER,st_step 100003_0,ADMINISTRATION,2018-02-07 08:58:46+00:00,11558.650000,2,2,8,693519.0,459154.0
12790,budget 86566,declaration number 100001,600.844121,declaration 100000,Declaration FINAL_APPROVED by SUPERVISOR,STAFF MEMBER,st_step 100002_0,SUPERVISOR,2018-02-08 09:59:05+00:00,1500.316667,2,3,9,783538.0,369135.0
12791,budget 86566,declaration number 100001,600.844121,declaration 100000,Request Payment,SYSTEM,dd_declaration 100000_19,UNDEFINED,2018-02-09 11:42:49+00:00,1543.733333,2,4,11,876162.0,276511.0
12792,budget 86566,declaration number 100001,600.844121,declaration 100000,Payment Handled,SYSTEM,dd_declaration 100000_20,UNDEFINED,2018-02-12 16:31:20+00:00,4608.516667,2,0,16,1152673.0,0.0


In [235]:
data[timestamp_col].min()

Timestamp('2017-01-09 08:49:50+0000', tz='UTC')

In [19]:
# impute missing values
grouped = data.sort_values(timestamp_col, ascending=True, kind='mergesort').groupby(case_id_col)
for col in static_cols + dynamic_cols:
    data[col] = grouped[col].transform(lambda grp: grp.fillna(method='ffill'))

data[cat_cols] = data[cat_cols].fillna('missing')
data = data.fillna(0)

In [20]:
# set infrequent factor levels to "other"
for col in cat_cols:
    counts = data[col].value_counts()
    mask = data[col].isin(counts[counts >= freq_threshold].index)
    data.loc[~mask, col] = "other"

In [24]:
data.head()

,BudgetNumber,DeclarationNumber,Amount,Case ID,Activity,Resource,id,Role,Complete Timestamp,duration,month,weekday,hour,elapsed,remtime
12788,budget 86566,other,600.844121,declaration 100000,Declaration SUBMITTED by EMPLOYEE,STAFF MEMBER,other,EMPLOYEE,2018-01-30 08:20:07+00:00,0.000000,1,1,8,0.0,1152673.0
12789,budget 86566,other,600.844121,declaration 100000,Declaration APPROVED by ADMINISTRATION,STAFF MEMBER,other,ADMINISTRATION,2018-02-07 08:58:46+00:00,11558.650000,2,2,8,693519.0,459154.0
12790,budget 86566,other,600.844121,declaration 100000,Declaration FINAL_APPROVED by SUPERVISOR,STAFF MEMBER,other,SUPERVISOR,2018-02-08 09:59:05+00:00,1500.316667,2,3,9,783538.0,369135.0
12791,budget 86566,other,600.844121,declaration 100000,Request Payment,SYSTEM,other,UNDEFINED,2018-02-09 11:42:49+00:00,1543.733333,2,4,11,876162.0,276511.0
12792,budget 86566,other,600.844121,declaration 100000,Payment Handled,SYSTEM,other,UNDEFINED,2018-02-12 16:31:20+00:00,4608.516667,2,0,16,1152673.0,0.0


In [25]:
data.to_csv(os.path.join(output_data_folder,filename), sep=";", index=False)

# Trial

In [242]:
filename = "../experiments/logdata/bpi20.csv"

case_id_col = "Case ID"
activity_col = "Activity"
timestamp_col = "Complete Timestamp"
label_col = "remtime"
pos_label= "regular"
neg_label = "deviant"

# features for classifier
dynamic_cat_cols = ["Activity", "Resource","id","month", "weekday", "hour"]
static_cat_cols = [ "BudgetNumber", "DeclarationNumber"]
dynamic_num_cols = [ "duration","elapsed" ]
static_num_cols = ["Amount"]


In [248]:
def read_dataset():
    # read dataset
    dtypes = {col:"object" for col in dynamic_cat_cols+static_cat_cols+[case_id_col, timestamp_col]}
    for col in dynamic_num_cols + static_num_cols:
        dtypes[col] = "float"

    dtypes[label_col] = "float"  # remaining time should be float


    data = pd.read_csv(filename, sep=";", dtype=dtypes)
    data[timestamp_col] = pd.to_datetime(data[timestamp_col])

    # segment_indices = pd.read_csv("logdata/relevant_case_id_indices.csv")["Case ID"]
    # indexes = set(segment_indices)
    #
    # data = data[data[self.case_id_col].isin(indexes)]

    return data


def split_data(data, train_ratio):  
    # split into train and test using temporal split

    grouped = data.groupby(case_id_col)
    start_timestamps = grouped[timestamp_col].min().reset_index()
    start_timestamps = start_timestamps.sort_values(timestamp_col, ascending=True, kind='mergesort')
    train_ids = list(start_timestamps[case_id_col])[:int(train_ratio*len(start_timestamps))]
    train = data[data[case_id_col].isin(train_ids)].sort_values(timestamp_col, ascending=True, kind='mergesort')
    test = data[~data[case_id_col].isin(train_ids)].sort_values(timestamp_col, ascending=True, kind='mergesort')

    return (train, test)


In [245]:
# read the data
data = read_dataset()
data.head()

,BudgetNumber,DeclarationNumber,Amount,Case ID,Activity,Resource,id,Complete Timestamp,duration,month,weekday,hour,elapsed,remtime
0,budget 86566,declaration number 100001,600.844121,declaration 100000,Declaration SUBMITTED by EMPLOYEE,STAFF MEMBER,st_step 100004_0,2018-01-30 08:20:07+00:00,0.000000,1,1,8,0.0,1152673.0
1,budget 86566,declaration number 100001,600.844121,declaration 100000,Declaration APPROVED by ADMINISTRATION,STAFF MEMBER,st_step 100003_0,2018-02-07 08:58:46+00:00,11558.650000,2,2,8,693519.0,459154.0
2,budget 86566,declaration number 100001,600.844121,declaration 100000,Declaration FINAL_APPROVED by SUPERVISOR,STAFF MEMBER,st_step 100002_0,2018-02-08 09:59:05+00:00,1500.316667,2,3,9,783538.0,369135.0
3,budget 86566,declaration number 100001,600.844121,declaration 100000,Request Payment,SYSTEM,dd_declaration 100000_19,2018-02-09 11:42:49+00:00,1543.733333,2,4,11,876162.0,276511.0
4,budget 86566,declaration number 100001,600.844121,declaration 100000,Payment Handled,SYSTEM,dd_declaration 100000_20,2018-02-12 16:31:20+00:00,4608.516667,2,0,16,1152673.0,0.0


In [249]:
# split data into train and test
train, _ = split_data(data, 0.8)

In [250]:
train.shape

(44761, 14)